In [1]:
from nltk.tokenize import RegexpTokenizer
import cPickle as pickle
import string
import os

%matplotlib inline
import numpy as np
import random
import time

import logging
from logging import info

from multiprocessing import Pool as ThreadPool
import itertools

In [2]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [3]:
SAMPLE_RATIO = 0.01

In [4]:
root_location = "/big/s/shalaby/"
exports_location = root_location + "exported_data/"

#training_file = root_location + 'docs_output_training_validation_documents_' + str(SAMPLE_RATIO)
training_file = root_location + 'docs_output.json'

doc_classifications_map_file = exports_location + "doc_classification_map.pkl"
sections_file = exports_location + "sections.pkl"
classes_file = exports_location + "classes.pkl"
subclasses_file = exports_location + "subclasses.pkl"
classifications_output = exports_location + "classifications.pkl"
# training_docs_list_file = exports_location + "training_documents_" + str(SAMPLE_RATIO) + "_sample.pkl"
# validation_docs_list_file = exports_location + "validation_documents_" + str(SAMPLE_RATIO) + "_sample.pkl"
training_docs_list_file = exports_location + "training_docs_list.pkl"
validation_docs_list_file = exports_location + "validation_docs_list.pkl"
test_docs_list_file = exports_location + "test_docs_list.pkl"

In [5]:
%%time
doc_classification_map = pickle.load(open(doc_classifications_map_file))
sections = pickle.load(open(sections_file))
classes = pickle.load(open(classes_file))
subclasses = pickle.load(open(subclasses_file))
training_docs_list = pickle.load(open(training_docs_list_file))
validation_docs_list = pickle.load(open(validation_docs_list_file))
test_docs_list = pickle.load(open(test_docs_list_file))

CPU times: user 29.3 s, sys: 6.14 s, total: 35.4 s
Wall time: 37.5 s


In [6]:
len(training_docs_list)

1286325

In [7]:
len(training_docs_list) + len(validation_docs_list)

1607798

In [8]:
len(validation_docs_list)

321473

In [9]:
len(test_docs_list)

401877

In [15]:
MIN_SIZE = 0
NUMBER_INDICATOR = "number_inidicator"
CURRENCY_INDICATOR = "currency_inidicator"
CHEMICAL_INDICATOR = "chemical_inidicator"

In [16]:
def stemtokenizer(text):
    """ MAIN FUNCTION to get clean stems out of a text. A list of clean stems are returned """
    tokenizer = RegexpTokenizer(r'\s+', gaps=True)
    tokens = tokenizer.tokenize(text)
    stems = []  # result
    for token in tokens:
        stem = token.lower()
        stem = stem.strip(string.punctuation)
        if stem:
            if is_number(stem):
                stem = NUMBER_INDICATOR
            elif is_currency(stem):
                stem = CURRENCY_INDICATOR
            elif is_chemical(stem):
                stem = CHEMICAL_INDICATOR
            else:
                stem = stem.strip(string.punctuation)
            if stem and len(stem) >= MIN_SIZE:
                # extract uni-grams
                stems.append(stem)
    del tokens
    return stems

def is_number(str):
    """ Returns true if given string is a number (float or int)"""
    try:
        float(str.replace(",", ""))
        return True
    except ValueError:
        return False

def is_currency(str):
    return str[0] == "$"

def is_chemical(str):
    return str.count("-") > 3

## Get Separate Training and Validation Data Files

In [11]:
BATCH_SIZE = 10000
# TRAINING_PREPROCESSED_FILES_PREFIX = "/mnt/data2/shalaby/training_docs_sample_%s_data_preprocessed-" % str(SAMPLE_RATIO)
# TRAINING_PREPROCESSED_DOCIDS_FILES_PREFIX = "/mnt/data2/shalaby/training_docs_sample_%s_docids_preprocessed-" % str(SAMPLE_RATIO)
# VALIDATION_PREPROCESSED_FILES_PREFIX = "/mnt/data2/shalaby/validation_docs_sample_%s_data_preprocessed-" % str(SAMPLE_RATIO)
# VALIDATION_PREPROCESSED_DOCIDS_FILES_PREFIX = "/mnt/data2/shalaby/validation_docs_sample_%s_docids_preprocessed-" % str(SAMPLE_RATIO)

TRAINING_PREPROCESSED_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/training_docs_data_preprocessed-"
TRAINING_PREPROCESSED_DOCIDS_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/training_docs_docids_preprocessed-"
VALIDATION_PREPROCESSED_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/validation_docs_data_preprocessed-"
VALIDATION_PREPROCESSED_DOCIDS_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/validation_docs_docids_preprocessed-"
TEST_PREPROCESSED_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/test_docs_data_preprocessed-"
TEST_PREPROCESSED_DOCIDS_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/test_docs_docids_preprocessed-"


TRAINING_MERGED_PREPROCESSED_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/training_docs_merged_data_preprocessed-"
TRAINING_MERGED_PREPROCESSED_DOCIDS_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/training_docs_merged_docids_preprocessed-"
VALIDATION_MERGED_PREPROCESSED_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/validation_docs_merged_data_preprocessed-"
VALIDATION_MERGED_PREPROCESSED_DOCIDS_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/validation_docs_merged_docids_preprocessed-"
TEST_MERGED_PREPROCESSED_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/test_docs_merged_data_preprocessed-"
TEST_MERGED_PREPROCESSED_DOCIDS_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/test_docs_merged_docids_preprocessed-"

In [12]:
def write_batch(file_prefix, doc_files_prefix, batch_lines, doc_ids, batch_start):
    if len(batch_lines):
        print "writing batch %d" % batch_start
        with open(file_prefix + str(batch_start), 'w') as batch_file:
            for line in batch_lines:
                batch_file.write((u" ".join(line) + "\n").encode('utf-8'))
        pickle.dump(doc_ids, open(doc_files_prefix + str(batch_start), 'w'))


Load Line Positions

In [13]:
line_positions = pickle.load(open("/big/s/shalaby/exported_data/line_positions.pkl", "r"))

Compute Line Positions

In [9]:
%%time
line_positions = dict()
with open(training_file) as f:
    
    i = 0
    line_positions[i] = f.tell()
    line = f.readline()
    while line:
        i+=1
        if not line.strip(): continue
        line_positions[i] = f.tell()
        line = f.readline()
    del line_positions[i]

CPU times: user 3min 54s, sys: 30.6 s, total: 4min 25s
Wall time: 4min 53s


In [12]:
pickle.dump(line_positions, open("/big/s/shalaby/exported_data/line_positions.pkl", "w"))

In [14]:
len(line_positions)

2009750

# Training

This will create uneven batches of documents, we need to run the following step to make it even

In [32]:
def multithreaded_batch_creation(start_index):

    file_prefix = TRAINING_PREPROCESSED_FILES_PREFIX
    doc_file_prefix = TRAINING_PREPROCESSED_DOCIDS_FILES_PREFIX
    
    if os.path.exists(file_prefix + str(start_index)):
        info("Batch {} already exists, skipping..".format(start_index))
        return
    
    info("Batch creation working on {}\n".format(start_index))
    with open(training_file) as file_obj:
        file_obj.seek(line_positions[start_index])
        token_lines, doc_ids = [], []
        start_time = time.time()
        for i, line in enumerate(file_obj):
            (doc_id, text) = eval(line)
            if doc_id in training_docs_list:
                token_lines.append(stemtokenizer(text))
                doc_ids.append(doc_id)
                if len(token_lines) % 1000 == 0: info(len(token_lines))
            if i >= BATCH_SIZE:
                break
    duration = time.time() - start_time
    info("Finished batch of {:d} in {:.0f}m {:.0f}s".format(BATCH_SIZE, *divmod(duration, 60)))
    info("For index {}, the actual number of lines written is: {}".format(start_index, len(token_lines)))
    write_batch(file_prefix, doc_file_prefix, token_lines, doc_ids, start_index)
    del token_lines
    del doc_ids

In [ ]:
pool.close()
pool.terminate()

In [35]:
(divmod(len(line_positions), BATCH_SIZE)[0] +1) * BATCH_SIZE

2010000

In [36]:
pool = ThreadPool(10)
# +1 since range is end-exclusive
batches = range(0, (divmod(len(line_positions), BATCH_SIZE)[0] + 1) * BATCH_SIZE, BATCH_SIZE )
indices = pool.map(multithreaded_batch_creation, batches)

2017-01-03 20:05:40,516 : INFO : Batch 180000 already exists, skipping..
2017-01-03 20:05:40,517 : INFO : Batch 240000 already exists, skipping..
2017-01-03 20:05:40,517 : INFO : Batch 120000 already exists, skipping..
2017-01-03 20:05:40,517 : INFO : Batch 420000 already exists, skipping..
2017-01-03 20:05:40,517 : INFO : Batch 480000 already exists, skipping..
2017-01-03 20:05:40,521 : INFO : Batch 190000 already exists, skipping..
2017-01-03 20:05:40,522 : INFO : Batch 130000 already exists, skipping..
2017-01-03 20:05:40,522 : INFO : Batch 430000 already exists, skipping..
2017-01-03 20:05:40,517 : INFO : Batch 0 already exists, skipping..
2017-01-03 20:05:40,521 : INFO : Batch 60000 already exists, skipping..
2017-01-03 20:05:40,516 : INFO : Batch 540000 already exists, skipping..
2017-01-03 20:05:40,519 : INFO : Batch 360000 already exists, skipping..
2017-01-03 20:05:40,522 : INFO : Batch 490000 already exists, skipping..
2017-01-03 20:05:40,523 : INFO : Batch 200000 already exi

writing batch 2000000


### Join up the files we wrote to form BATCH_SIZE block files

In [50]:
curr_index = 0
curr_read_file_index = 0
curr_write_file_index = 0

read_file = None
write_file = None

def read_line():
    global read_file, curr_read_file_index
    while True:
        if read_file is None:
            if os.path.exists(TRAINING_PREPROCESSED_FILES_PREFIX + str(curr_read_file_index)):
                read_file = open(TRAINING_PREPROCESSED_FILES_PREFIX + str(curr_read_file_index), "r")
            else: 
                raise StopIteration()
        for line in read_file:
            yield line
            
        curr_read_file_index += BATCH_SIZE
        info("Reading new file for batch {}".format(curr_read_file_index))
        read_file.close()
        read_file = None

def write_line(line):
    global write_file, curr_write_file_index, curr_index
    if write_file is None:
        write_file = open(TRAINING_MERGED_PREPROCESSED_FILES_PREFIX + str(curr_write_file_index), "w")
    write_file.write(line)
    curr_index += 1
    if curr_index % BATCH_SIZE == 0:
        curr_write_file_index += BATCH_SIZE
        info("Writing to a new file for batch {}".format(curr_write_file_index))
        write_file.close()
        write_file = None
    

for line in read_line():
    write_line(line)

2017-01-04 02:25:14,695 : INFO : Reading new file for batch 10000
2017-01-04 02:25:16,752 : INFO : Writing to a new file for batch 10000
2017-01-04 02:25:27,313 : INFO : Reading new file for batch 20000
2017-01-04 02:25:30,973 : INFO : Reading new file for batch 30000
2017-01-04 02:25:31,467 : INFO : Writing to a new file for batch 20000
2017-01-04 02:25:44,787 : INFO : Reading new file for batch 40000
2017-01-04 02:25:48,578 : INFO : Writing to a new file for batch 30000
2017-01-04 02:25:59,947 : INFO : Reading new file for batch 50000
2017-01-04 02:26:04,004 : INFO : Reading new file for batch 60000
2017-01-04 02:26:05,102 : INFO : Writing to a new file for batch 40000
2017-01-04 02:26:17,433 : INFO : Reading new file for batch 70000
2017-01-04 02:26:20,341 : INFO : Writing to a new file for batch 50000
2017-01-04 02:26:31,509 : INFO : Reading new file for batch 80000
2017-01-04 02:26:36,285 : INFO : Reading new file for batch 90000
2017-01-04 02:26:37,758 : INFO : Writing to a new f

#### Rearranging Docids files

In [60]:
all_doc_ids = []

for i in range(0, 100000000, BATCH_SIZE):
    if os.path.exists(TRAINING_PREPROCESSED_DOCIDS_FILES_PREFIX + str(i)):
        doc_ids = pickle.load(open(TRAINING_PREPROCESSED_DOCIDS_FILES_PREFIX + str(i)))
        all_doc_ids.extend(doc_ids)
    else:
        break
        

for i in range(0, 100000000, BATCH_SIZE):
    if i < len(all_doc_ids):
        pickle.dump(all_doc_ids[i: i + BATCH_SIZE], open(TRAINING_MERGED_PREPROCESSED_DOCIDS_FILES_PREFIX + str(i), "w"))
    else:
        break

In [59]:
len(all_doc_ids)

1286458

# Validation

In [63]:
def multithreaded_batch_creation(start_index):

    file_prefix = VALIDATION_PREPROCESSED_FILES_PREFIX
    doc_file_prefix = VALIDATION_PREPROCESSED_DOCIDS_FILES_PREFIX
    
    if os.path.exists(file_prefix + str(start_index)):
        info("Batch {} already exists, skipping..".format(start_index))
        return
    
    info("Batch creation working on {}\n".format(start_index))
    with open(training_file) as file_obj:
        file_obj.seek(line_positions[start_index])
        token_lines, doc_ids = [], []
        start_time = time.time()
        for i, line in enumerate(file_obj):
            (doc_id, text) = eval(line)
            if doc_id in validation_docs_list:
                token_lines.append(stemtokenizer(text))
                doc_ids.append(doc_id)
                if len(token_lines) % 1000 == 0: info(len(token_lines))
            if i >= BATCH_SIZE:
                break
    duration = time.time() - start_time
    info("Finished batch of {:d} in {:.0f}m {:.0f}s".format(BATCH_SIZE, *divmod(duration, 60)))
    info("For index {}, the actual number of lines written is: {}".format(start_index, len(token_lines)))
    write_batch(file_prefix, doc_file_prefix, token_lines, doc_ids, start_index)
    del token_lines
    del doc_ids

In [64]:
pool = ThreadPool(10)
# +1 since range is end-exclusive
batches = range(0, (divmod(len(line_positions), BATCH_SIZE)[0] + 1) * BATCH_SIZE, BATCH_SIZE )
indices = pool.map(multithreaded_batch_creation, batches)

2017-01-04 04:12:22,405 : INFO : Batch creation working on 0

2017-01-04 04:12:22,419 : INFO : Batch creation working on 300000

2017-01-04 04:12:22,406 : INFO : Batch creation working on 180000

2017-01-04 04:12:22,431 : INFO : Batch creation working on 60000

2017-01-04 04:12:22,499 : INFO : Batch creation working on 120000

2017-01-04 04:12:22,503 : INFO : Batch creation working on 420000

2017-01-04 04:12:22,507 : INFO : Batch creation working on 480000

2017-01-04 04:12:22,499 : INFO : Batch creation working on 240000

2017-01-04 04:12:22,510 : INFO : Batch creation working on 540000

2017-01-04 04:12:22,499 : INFO : Batch creation working on 360000

2017-01-04 04:19:00,955 : INFO : 1000
2017-01-04 04:19:04,784 : INFO : 1000
2017-01-04 04:19:05,780 : INFO : 1000
2017-01-04 04:19:17,581 : INFO : 1000
2017-01-04 04:19:22,408 : INFO : 1000
2017-01-04 04:19:22,974 : INFO : 1000
2017-01-04 04:19:26,641 : INFO : 1000
2017-01-04 04:19:33,416 : INFO : 1000
2017-01-04 04:21:05,734 : INFO :

writing batch 540000


2017-01-04 04:23:16,265 : INFO : Finished batch of 10000 in 10m 54s
2017-01-04 04:23:16,273 : INFO : For index 180000, the actual number of lines written is: 1556


writing batch 180000


2017-01-04 04:23:17,067 : INFO : Finished batch of 10000 in 10m 55s
2017-01-04 04:23:17,070 : INFO : For index 360000, the actual number of lines written is: 1637


...

writing batch 1850000


2017-01-04 09:05:09,293 : INFO : Finished batch of 10000 in 13m 60s
2017-01-04 09:05:09,298 : INFO : For index 1910000, the actual number of lines written is: 1606


writing batch 1910000


2017-01-04 09:06:38,757 : INFO : Finished batch of 10000 in 14m 10s
2017-01-04 09:06:38,762 : INFO : For index 1970000, the actual number of lines written is: 1635


writing batch 1970000


#### Join up the files we wrote to form BATCH_SIZE block files

In [65]:
curr_index = 0
curr_read_file_index = 0
curr_write_file_index = 0

read_file = None
write_file = None

def read_line():
    global read_file, curr_read_file_index
    while True:
        if read_file is None:
            if os.path.exists(VALIDATION_PREPROCESSED_FILES_PREFIX + str(curr_read_file_index)):
                read_file = open(VALIDATION_PREPROCESSED_FILES_PREFIX + str(curr_read_file_index), "r")
            else: 
                raise StopIteration()
        for line in read_file:
            yield line
            
        curr_read_file_index += BATCH_SIZE
        info("Reading new file for batch {}".format(curr_read_file_index))
        read_file.close()
        read_file = None

def write_line(line):
    global write_file, curr_write_file_index, curr_index
    if write_file is None:
        write_file = open(VALIDATION_MERGED_PREPROCESSED_FILES_PREFIX + str(curr_write_file_index), "w")
    write_file.write(line)
    curr_index += 1
    if curr_index % BATCH_SIZE == 0:
        curr_write_file_index += BATCH_SIZE
        info("Writing to a new file for batch {}".format(curr_write_file_index))
        write_file.close()
        write_file = None
    

for line in read_line():
    write_line(line)

2017-01-04 09:06:46,449 : INFO : Reading new file for batch 10000
2017-01-04 09:06:47,245 : INFO : Reading new file for batch 20000
2017-01-04 09:06:48,032 : INFO : Reading new file for batch 30000
2017-01-04 09:06:48,855 : INFO : Reading new file for batch 40000
2017-01-04 09:06:49,657 : INFO : Reading new file for batch 50000
2017-01-04 09:06:50,486 : INFO : Reading new file for batch 60000
2017-01-04 09:06:50,819 : INFO : Writing to a new file for batch 10000
2017-01-04 09:06:59,380 : INFO : Reading new file for batch 70000
2017-01-04 09:07:00,196 : INFO : Reading new file for batch 80000
2017-01-04 09:07:01,050 : INFO : Reading new file for batch 90000
2017-01-04 09:07:02,035 : INFO : Reading new file for batch 100000
2017-01-04 09:07:02,900 : INFO : Reading new file for batch 110000
2017-01-04 09:07:03,702 : INFO : Reading new file for batch 120000
2017-01-04 09:07:04,183 : INFO : Writing to a new file for batch 20000
2017-01-04 09:07:12,708 : INFO : Reading new file for batch 130

#### Rearranging Docids files

In [66]:
all_doc_ids = []

for i in range(0, 100000000, BATCH_SIZE):
    if os.path.exists(VALIDATION_PREPROCESSED_DOCIDS_FILES_PREFIX + str(i)):
        doc_ids = pickle.load(open(VALIDATION_PREPROCESSED_DOCIDS_FILES_PREFIX + str(i)))
        all_doc_ids.extend(doc_ids)
    else:
        break
        

for i in range(0, 100000000, BATCH_SIZE):
    if i < len(all_doc_ids):
        pickle.dump(all_doc_ids[i: i + BATCH_SIZE], open(VALIDATION_MERGED_PREPROCESSED_DOCIDS_FILES_PREFIX + str(i), "w"))
    else:
        break

# Test Data

In [18]:
def multithreaded_batch_creation(start_index):

    file_prefix = TEST_PREPROCESSED_FILES_PREFIX
    doc_file_prefix = TEST_PREPROCESSED_DOCIDS_FILES_PREFIX
    
    if os.path.exists(file_prefix + str(start_index)):
        info("Batch {} already exists, skipping..".format(start_index))
        return
    
    info("Batch creation working on {}\n".format(start_index))
    with open(training_file) as file_obj:
        file_obj.seek(line_positions[start_index])
        token_lines, doc_ids = [], []
        start_time = time.time()
        for i, line in enumerate(file_obj):
            (doc_id, text) = eval(line)
            if doc_id in test_docs_list:
                token_lines.append(stemtokenizer(text))
                doc_ids.append(doc_id)
                if len(token_lines) % 1000 == 0: info(len(token_lines))
            if i >= BATCH_SIZE:
                break
    duration = time.time() - start_time
    info("Finished batch of {:d} in {:.0f}m {:.0f}s".format(BATCH_SIZE, *divmod(duration, 60)))
    info("For index {}, the actual number of lines written is: {}".format(start_index, len(token_lines)))
    write_batch(file_prefix, doc_file_prefix, token_lines, doc_ids, start_index)
    del token_lines
    del doc_ids

In [17]:
(divmod(len(line_positions), BATCH_SIZE)[0] +1) * BATCH_SIZE

2010000

In [19]:
pool = ThreadPool(16)
# +1 since range is end-exclusive
batches = range(0, (divmod(len(line_positions), BATCH_SIZE)[0] + 1) * BATCH_SIZE, BATCH_SIZE )
indices = pool.map(multithreaded_batch_creation, batches)

2017-01-30 18:34:43,473 : INFO : Batch creation working on 40000

2017-01-30 18:34:43,473 : INFO : Batch creation working on 400000

2017-01-30 18:34:43,474 : INFO : Batch creation working on 240000

2017-01-30 18:34:43,473 : INFO : Batch creation working on 0

2017-01-30 18:34:43,473 : INFO : Batch creation working on 200000

2017-01-30 18:34:43,474 : INFO : Batch creation working on 440000

2017-01-30 18:34:43,474 : INFO : Batch creation working on 360000

2017-01-30 18:34:43,477 : INFO : Batch creation working on 120000

2017-01-30 18:34:43,477 : INFO : Batch creation working on 80000

2017-01-30 18:34:43,477 : INFO : Batch creation working on 480000

2017-01-30 18:34:43,478 : INFO : Batch creation working on 280000

2017-01-30 18:34:43,479 : INFO : Batch creation working on 320000

2017-01-30 18:34:43,479 : INFO : Batch creation working on 160000

2017-01-30 18:34:43,479 : INFO : Batch creation working on 520000

2017-01-30 18:34:43,482 : INFO : Batch creation working on 560000

20

writing batch 0


2017-01-30 18:48:07,244 : INFO : Batch creation working on 10000

2017-01-30 18:48:18,067 : INFO : 2000
2017-01-30 18:48:19,040 : INFO : 2000
2017-01-30 18:48:26,435 : INFO : 2000
2017-01-30 18:48:41,425 : INFO : Finished batch of 10000 in 13m 58s
2017-01-30 18:48:41,430 : INFO : For index 40000, the actual number of lines written is: 2029


writing batch 40000


2017-01-30 18:48:43,648 : INFO : Finished batch of 10000 in 13m 60s
2017-01-30 18:48:43,651 : INFO : For index 200000, the actual number of lines written is: 1977


...

writing batch 1980000


2017-01-30 22:05:12,400 : INFO : Batch creation working on 1990000

2017-01-30 22:07:15,335 : INFO : 2000
2017-01-30 22:07:35,957 : INFO : Finished batch of 10000 in 11m 36s
2017-01-30 22:07:35,959 : INFO : For index 1950000, the actual number of lines written is: 2054


writing batch 1950000


2017-01-30 22:10:05,701 : INFO : 1000
2017-01-30 22:15:04,340 : INFO : 2000
2017-01-30 22:15:11,871 : INFO : Finished batch of 10000 in 9m 59s
2017-01-30 22:15:11,872 : INFO : For index 1990000, the actual number of lines written is: 2022


writing batch 1990000


In [22]:
pool.close()
pool.terminate()

### Join up the files we wrote to form BATCH_SIZE block files

In [ ]:
curr_index = 0
curr_read_file_index = 0
curr_write_file_index = 0

read_file = None
write_file = None

def read_line():
    global read_file, curr_read_file_index
    while True:
        if read_file is None:
            if os.path.exists(TEST_PREPROCESSED_FILES_PREFIX + str(curr_read_file_index)):
                read_file = open(TEST_PREPROCESSED_FILES_PREFIX + str(curr_read_file_index), "r")
            else: 
                raise StopIteration()
        for line in read_file:
            yield line
            
        curr_read_file_index += BATCH_SIZE
        info("Reading new file for batch {}".format(curr_read_file_index))
        read_file.close()
        read_file = None

def write_line(line):
    global write_file, curr_write_file_index, curr_index
    if write_file is None:
        write_file = open(TEST_MERGED_PREPROCESSED_FILES_PREFIX + str(curr_write_file_index), "w")
    write_file.write(line)
    curr_index += 1
    if curr_index % BATCH_SIZE == 0:
        curr_write_file_index += BATCH_SIZE
        info("Writing to a new file for batch {}".format(curr_write_file_index))
        write_file.close()
        write_file = None
    

for line in read_line():
    write_line(line)

2017-01-30 22:28:10,633 : INFO : Reading new file for batch 10000
2017-01-30 22:28:51,119 : INFO : Reading new file for batch 20000
2017-01-30 22:28:59,217 : INFO : Reading new file for batch 30000
2017-01-30 22:29:04,416 : INFO : Reading new file for batch 40000
2017-01-30 22:29:07,508 : INFO : Writing to a new file for batch 10000
2017-01-30 22:29:16,125 : INFO : Reading new file for batch 50000
2017-01-30 22:29:18,380 : INFO : Reading new file for batch 60000
2017-01-30 22:29:20,447 : INFO : Reading new file for batch 70000
2017-01-30 22:29:22,347 : INFO : Reading new file for batch 80000
2017-01-30 22:29:23,949 : INFO : Reading new file for batch 90000
2017-01-30 22:29:25,618 : INFO : Writing to a new file for batch 20000


#### Rearranging the Docids

In [21]:
all_doc_ids = []

for i in range(0, 100000000, BATCH_SIZE):
    if os.path.exists(TEST_PREPROCESSED_DOCIDS_FILES_PREFIX + str(i)):
        doc_ids = pickle.load(open(TEST_PREPROCESSED_DOCIDS_FILES_PREFIX + str(i)))
        all_doc_ids.extend(doc_ids)
    else:
        break
        

for i in range(0, 100000000, BATCH_SIZE):
    if i < len(all_doc_ids):
        pickle.dump(all_doc_ids[i: i + BATCH_SIZE], open(TEST_MERGED_PREPROCESSED_DOCIDS_FILES_PREFIX + str(i), "w"))
    else:
        break

## Concatenating 10,000 batch files to 100,000 batch files

In [70]:
%%time
write_file = open("/big/s/shalaby/preprocessed_data/training_large_preprocessed-0", "w")
for i in range(0, 100000, BATCH_SIZE):
    with open(TRAINING_MERGED_PREPROCESSED_FILES_PREFIX + str(i)) as read_file:
        %time content = read_file.readlines()
    write_file.writelines(content)
    print i
write_file.close()

CPU times: user 0 ns, sys: 680 ms, total: 680 ms
Wall time: 755 ms
0
CPU times: user 0 ns, sys: 724 ms, total: 724 ms
Wall time: 918 ms
10000
CPU times: user 0 ns, sys: 328 ms, total: 328 ms
Wall time: 593 ms
20000
CPU times: user 0 ns, sys: 480 ms, total: 480 ms
Wall time: 968 ms
30000
CPU times: user 0 ns, sys: 968 ms, total: 968 ms
Wall time: 58 s
40000
CPU times: user 0 ns, sys: 664 ms, total: 664 ms
Wall time: 9.68 s
50000
CPU times: user 0 ns, sys: 736 ms, total: 736 ms
Wall time: 23.1 s
60000
CPU times: user 0 ns, sys: 712 ms, total: 712 ms
Wall time: 9.25 s
70000
CPU times: user 0 ns, sys: 612 ms, total: 612 ms
Wall time: 8.45 s
80000
CPU times: user 0 ns, sys: 756 ms, total: 756 ms
Wall time: 28.8 s
90000
CPU times: user 0 ns, sys: 12.2 s, total: 12.2 s
Wall time: 2min 37s


## Old Single Threaded Method

In [ ]:
%%time

batch_index = 13
file_prefix = TRAINING_PREPROCESSED_FILES_PREFIX
doc_file_prefix = TRAINING_PREPROCESSED_DOCIDS_FILES_PREFIX
with open(training_file) as file_obj:
    token_lines, doc_ids = [], []
    start_time = time.time()
    for index,line in enumerate(file_obj):
        if index < 130000: continue
        (doc_id, text) = eval(line)
        if doc_id in training_docs_list:
            token_lines.append(stemtokenizer(text))
            doc_ids.append(doc_id)
            if len(token_lines) % 1000 == 0: info(len(token_lines))
            if len(token_lines) % BATCH_SIZE == 0:
                duration = time.time() - start_time
                info("Finished batch of {:d} in {:.0f}m {:.0f}s".format(BATCH_SIZE, *divmod(duration, 60)))
                start_time = time.time()
                write_batch(file_prefix, doc_file_prefix, token_lines, doc_ids, batch_index * BATCH_SIZE)
                batch_index += 1
                token_lines, doc_ids = [], []
    duration = time.time() - start_time
    info("Finished batch of {:d} in {:d}m {:.0f}s".format(BATCH_SIZE, *divmod(duration, 60)))
    write_batch(file_prefix, doc_file_prefix, token_lines, doc_ids, batch_index * BATCH_SIZE)

Validation

In [8]:
%%time

batch_index = 0
file_prefix = VALIDATION_PREPROCESSED_FILES_PREFIX
doc_file_prefix = VALIDATION_PREPROCESSED_DOCIDS_FILES_PREFIX
with open(training_file) as file_obj:
    token_lines, doc_ids = [], []
    for line in file_obj:
        (doc_id, text) = eval(line)
        if doc_id in validation_docs_list:
            token_lines.append(stemtokenizer(text))
            doc_ids.append(doc_id)
            if len(token_lines) % BATCH_SIZE == 0:
                %time write_batch(file_prefix, doc_file_prefix, token_lines, doc_ids, batch_index * BATCH_SIZE)
                batch_index += 1
                token_lines, doc_ids = [], []
    write_batch(file_prefix, doc_file_prefix, token_lines, doc_ids, batch_index * BATCH_SIZE)

writing batch 0
CPU times: user 42.5 s, sys: 832 ms, total: 43.4 s
Wall time: 43.5 s


In [34]:
%%time
line_tokens = []
with open(TRAINING_PREPROCESSED_FILES_PREFIX + str(0)) as preproc_file:
    line_lengths = []
    for line in preproc_file:
        line_lengths.append(len(line))
        line_tokens.append(line.split(" "))

CPU times: user 1.53 s, sys: 856 ms, total: 2.38 s
Wall time: 2.39 s


### Old Method

we used to write the doc id and the tokens as tuples, then do an eval on them in reading time, but this turned out to be very slow

Training

In [ ]:
%%time
def write_batch(file_prefix, batch_lines, batch_start):
    if len(batch_lines):
        print "writing batch %d" % batch_start
        %time pickle.dump(batch_lines, open(file_prefix + str(batch_start), 'w'))
#         with open(file_prefix + str(batch_start), 'w') as batch_file:
#             for line in batch_lines:
#                 batch_file.write(str(line) + "\n")

batch_index = 0
file_prefix = TRAINING_PREPROCESSED_FILES_PREFIX
with open(training_file) as file_obj:
    token_lines = []
    for line in file_obj:
        (doc_id, text) = eval(line)
        if doc_id in training_docs_list:
            token_lines.append((doc_id, stemtokenizer(text)))
            if len(token_lines) % BATCH_SIZE == 0:
                write_batch(file_prefix, token_lines, batch_index * BATCH_SIZE)
                batch_index += 1
                token_lines = []
    write_batch(file_prefix, token_lines, batch_index * BATCH_SIZE)

Validation

In [8]:
batch_index = 0
file_prefix = VALIDATION_PREPROCESSED_FILES_PREFIX
with open(training_file) as file_obj:
    token_lines = []
    for line in file_obj:
        (doc_id, text) = eval(line)
        if doc_id in validation_docs_list:
            token_lines.append((doc_id, stemtokenizer(text)))
            if len(token_lines) % BATCH_SIZE == 0:
                write_batch(file_prefix, token_lines, batch_index * BATCH_SIZE)
                batch_index += 1
                token_lines = []
    write_batch(file_prefix, token_lines, batch_index * BATCH_SIZE)

writing batch 0
writing batch 10000


In [22]:
%%time
line_tokens = []
with open(TRAINING_PREPROCESSED_FILES_PREFIX + str(0)) as preproc_file:
    for line in preproc_file:
        line_tokens.append(line.split(" "))

CPU times: user 5 s, sys: 1.2 s, total: 6.19 s
Wall time: 6.18 s


In [24]:
line_tokens[0][:10]

['technical',
 'field',
 'the',
 'present',
 'invention',
 'generally',
 'relates',
 'to',
 'wireless',
 'communications']